In [ ]:
import pandas as pd
import numpy as np
import glob
from pyts.image import RecurrencePlot, GramianAngularField, MarkovTransitionField
from sklearn.metrics import accuracy_score
import gc
from modified_originial_attlstm import *

In [ ]:
# List files

files = glob.glob('data.*')
files_train = files[:-2]
files_test = files[1:-1]

In [ ]:
# Training / Testing batch

columns = ['time', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'b8', 'b9',
       'b10', 'b11', 'b12', 'b13', 'b14', 'b15', 'b16', 'b17', 'b18', 'b19',
       'b20', 'b21', 'b22', 'b23', 'b24', 'b25', 'b26', 'b27', 'b28', 'b29',
       'b30', 'b31', 'b32', 'b33', 'b34', 'b35', 'b36', 'b37', 'b38', 'b39',
       'b40', 'b41', 's1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9',
       's10', 's11', 's12', 's13', 's14', 's15', 's16', 's17', 's18', 's19',
       's20', 's21', 's22', 's23', 's24', 's25', 's26', 's27', 's28', 's29',
       's30', 's31', 's32', 's33', 's34', 's35', 's36', 's37', 's38', 's39',
       's40', 's41', 's42', 's43', 's44', 's45', 's46', 's47', 's48', 's49',
       's50', 'y_1m', 'y_5m', 'y_10m', 'y_15m', 'y_30m', 'y_hat1', 'y_hat2',
       'y_hat3']
for i in range(len(files_train)):
    #i=0
    #File loading
    if i == 0:
        df = pd.read_csv(files_train[i])
        df.columns = columns
    else:
        df = pd.read_csv(files_train[i],names=columns)
    test = pd.read_csv(files_test[i],header=None,names=columns)

    #Preprocessing
    df = df.set_index(pd.DatetimeIndex(pd.to_datetime(df[df.columns[0]])))
    target = df.y_10m
    df = df.drop(['s19','s24','time',"y_1m","y_5m","y_10m","y_15m","y_30m","y_hat1","y_hat2","y_hat3"],axis=1)
    df = df.fillna(0)

    test = test.set_index(pd.DatetimeIndex(pd.to_datetime(test[test.columns[0]])))
    target_test = test.y_10m
    test = test.drop(['s19','s24','time',"y_1m","y_5m","y_10m","y_15m","y_30m","y_hat1","y_hat2","y_hat3"],axis=1)
    test = test.fillna(0)
    test2 = np.array(test)

    #TRAINING
    batchsize = 128
    nhidden_encoder = 128
    nhidden_decoder = 128
    ntimestep = 2
    lr = 0.001
    epochs = 50
    model = DA_RNN(
        np.array(df),
        np.array(target),
        ntimestep,
        nhidden_encoder,
        nhidden_decoder,
        batchsize,
        lr,
        epochs,
        parallel=True
    )
    print("==> Start training ...")
    model.train()
    #TESTING
    y_pred = model.test(test2,target_test)
    #print("accuracy_score" + str(accuracy_score(np.sign(np.array(target_test)),np.sign(y_pred))))
    lin = np.linspace(0,1,5)
    print(pd.DataFrame(y_pred).quantile(lin))
    print("cor: " + str(np.corrcoef(target_test,y_pred)[0,1]))
    pd.DataFrame({'index':test.index,'y_pred':y_pred,'target_test':target_test}).to_csv("attlstm" + str(i) + ".csv")

In [ ]:
# Concatenate predictions

files = glob.glob('attlstm*')[1:]
for i,file in enumerate(files):
    df = pd.read_csv(file)
    df = df.set_index(pd.DatetimeIndex(pd.to_datetime(df[df.columns[0]]))) 
    df =df.y_pred
    if i == 0:
        all_df = df
    else:
        all_df = pd.concat((all_df,df))
all_df.to_csv('all_attlstm.csv')

In [ ]:
#NO NEED EVERYTHING BELOW

files = glob.glob('results/attlstm*')[1:]
for i,file in enumerate(files):
    df = pd.read_csv(file)
    df = df.set_index(pd.DatetimeIndex(pd.to_datetime(df[df.columns[0]]))) 
    df =df.y_pred
    if i == 0:
        all_df = df
    else:
        all_df = pd.concat((all_df,df))
all_df.to_csv('all_xg.csv')

In [ ]:
def minMax(x):
    returnurn pd.Series(index=['min','max'],data=[x.min(),x.max()])
minMax = df.apply(minMax).transpose()

In [ ]:
columns = ['time', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'b8', 'b9',
       'b10', 'b11', 'b12', 'b13', 'b14', 'b15', 'b16', 'b17', 'b18', 'b19',
       'b20', 'b21', 'b22', 'b23', 'b24', 'b25', 'b26', 'b27', 'b28', 'b29',
       'b30', 'b31', 'b32', 'b33', 'b34', 'b35', 'b36', 'b37', 'b38', 'b39',
       'b40', 'b41', 's1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9',
       's10', 's11', 's12', 's13', 's14', 's15', 's16', 's17', 's18', 's19',
       's20', 's21', 's22', 's23', 's24', 's25', 's26', 's27', 's28', 's29',
       's30', 's31', 's32', 's33', 's34', 's35', 's36', 's37', 's38', 's39',
       's40', 's41', 's42', 's43', 's44', 's45', 's46', 's47', 's48', 's49',
       's50', 'y_1m', 'y_5m', 'y_10m', 'y_15m', 'y_30m', 'y_hat1', 'y_hat2',
       'y_hat3']
df = pd.read_csv(files_train[0])
test = pd.read_csv(files_test[0],header=None,names=columns)

In [ ]:
df = df.set_index(pd.DatetimeIndex(pd.to_datetime(df[df.columns[0]])))
target = df.y_10m
df = df.drop(['s19','s24','Unnamed: 0',"y_1m","y_5m","y_10m","y_15m","y_30m","y_hat1","y_hat2","y_hat3"],axis=1)
df = df.fillna(0)
df2 = np.array(df.tail(10000))
test = test.set_index(pd.DatetimeIndex(pd.to_datetime(test[test.columns[0]])))
target_test = test.y_10m
test = test.drop(['s19','s24','time',"y_1m","y_5m","y_10m","y_15m","y_30m","y_hat1","y_hat2","y_hat3"],axis=1)
test = test.fillna(0)
test2 = np.array(test)

In [ ]:
transformer = RecurrencePlot()
df_temp1 = transformer.fit_transform(df2)

In [ ]:
transformer2 = GramianAngularField()
df_temp2 = transformer2.fit_transform(df2)

In [ ]:
transformer3 = MarkovTransitionField()
df_temp3 = transformer3.fit_transform(df2)

In [ ]:
df2 = (df_temp1+df_temp2+df_temp3)/3
df2 = np.array(df2).reshape(len(df2),df2.shape[1] * df2.shape[1])
df2.shape

In [ ]:
import xgboost as xg
model = xg.XGBRegressor(verbose=2,gpu_id=0,nthread=8)
model.fit(np.array(df2[-1000:]),np.array(target.tail(1000)))

In [ ]:
test2 = transformer.transform(test[10000:20000])

df2 = (df_temp1+df_temp2+df_temp3)/3
test2 = np.array(test2).reshape(len(test2),test2.shape[1] * test2.shape[1])
y_pred = model.predict(test2)
print("accuracy_score" + str(accuracy_score(np.sign(np.array(target_test[10000:20000])),np.sign(y_pred))))
#lin = np.linspace(0,1,101)
#print(pd.DataFrame(y_pred).quantile(lin))
print("cor" + str(np.corrcoef(target_test[10000:20000],y_pred)[0,1]))
pd.DataFrame({'index':test.index[10000:20000],'y_pred':y_pred,'target_test':target_test[10000:20000]}).to_csv("xg" + str(0) + ".csv")